In [1]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


https://www.psycopg.org/docs/
http://localhost:8082

Télécharger le pilote de H2**Relancez le kernel**: Kernel -> Restart kernel ...

Télécharger le pilote de H2

In [ ]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

In [2]:
import psycopg2
import os
local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

['jo8115.mv.db', '.ipynb_checkpoints', 'jo5115.mv.db']

## Démarrer le serveur SQL H2

In [ ]:
#Remplacez les valeurs des variables suivantes
#NN= numéro de groupe: chaine de caractères (ex: 01, 02,.., 13), remplacer xx 
NN = 'xx'
#BB=numero binome: chaîne de caractères (assigné par l'enseignant, ex: 01,..., 15), remplacer xx
BB = 'xx'
port = 5000+int(NN+BB)
print(f'Le numero du port utilisé pour la connexion à la BD est: {port}')

In [4]:
%%bash --bg --out output -s "$port"
java -Dh2.bindAddress=127.0.0.1 -cp h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort $1 -baseDir ./data -ifNotExists

# Fonctions utiles

## connect_H2

In [5]:
def connect_H2(db,user,password):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

## execute

In [12]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount:
            names = [desc[0] for desc in cursor.description]
            for attr in names:
                print(str(attr[:16]).ljust(16),end='|')
            print()
            for attr in names:
                print(''.ljust(17,'-'),end='')
            print()
            for ligne in cursor:
                for attr in ligne:
                    print(str(attr)[:16].ljust(16),end='|')
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

**TD 5 et TME5 : JOINTURES ET IMBRICATION AVEC IN ET EXISTS**

Ce TD/TME utilise les données contenues dans le fichier **bd_jo_v2_H2.sql**


**Attention**: vous pouvez cliquer sur les 3 lignes dans la fenêtre de gauche pour d'afficher les différentes sections du notebook   

# TD5 

On considère le schéma de la base JEUXOLYMPIQUE2014 qui décrit les athlètes et leurs résultats aux
épreuves des sports des Jeux Olympiques d'Hiver Sotchi 2014 :

**PAYS** ( <u>CODEPAYS</u>, NOMP)<br/>
Ex. ('FRA', 'France')<br/>
**SPORT** ( <u>SID</u>, NOMSP)<br/>
Ex. (1, 'Biathlon')<br/>
**EPREUVE** ( <u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN)<br/>
Ex. (10, 1, 'relais 4x7,5km', 'Hommes', 22/02/2014, 22/02/2014)<br/>
**ATHLETE** ( <u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)<br/>
Ex. (1000, 'SOBOLEV', 'Alexey', NULL, 'RUS')<br/>
**EQUIPE** ( <u>EQID</u>, CODEPAYS*)<br/>
Ex. (30, 'SUI')<br/>
**ATHLETESEQUIPE** ( <u>EQID*, AID*</u>)<br/>
Ex. (30, 796) : L'athlète (aid=796) a participé à l'équipe (eqid=30)<br/>
**RANGINDIVIDUEL** ( <u>EPID*, AID*</u>, RANG)<br/>
Ex. (15, 61, 1) : L'athlète (aid=61) a gagné la médaille d'or (rang=1) de l'épreuve (epid=15)<br/>
**RANGEQUIPE** ( <u>EPID*, EQID*</u>, RANG)<br/>
Ex. (10, 30, 14) : L'équipe (eqid=30) a été classée 14e à l'épreuve (epid=10)<br/>


Les attributs qui forment la clé primaire de chaque relation sont soulignés. Les clés étrangères sont
signalées avec une *. Les attributs aid, epid, eqid et sid correspondent aux identifiants des athlètes,
épreuves, équipes et sports et sont utilisés à la fois comme clé primaire ou comme référence (clé
étrangère) vers la relation correspondante.
La relation **PAYS** contient le code et le nom de tous les pays, même si ils n'ont pas participé aux
Jeux Olympiques. Les sports (n-uplets de la relation **SPORT**) sont un ensemble d'épreuves (n-uplets
de la relation **EPREUVE**). Pour chaque épreuve on connaît son nom et les date de début et fin de
l'épreuve. Les épreuves peuvent être individuelles ou par équipe. Dans le premier cas, la
participation des athlètes (n-uplets de la relation ATHLETE) est stocké dans la table
**RANGINDIVIDUEL** qui contient en plus le rang qu'ils ont obtenu (1 pour la médaille d'or). Pour les
épreuves par équipe les résultats sont stockés dans la relation **RANGEQUIPE**, alors que l'information
sur le pays de chaque équipe et ses participants et stocké dans les relations **EQUIPE** et
**ATHLETESEQUIPE**. Dans les relations **RANGINDIVIDUEL** et **RANGEQUIPE** l'attribut rang est égal à
null si l'athlète ou l'équipe a été disqualifié.


## Connexion à H2

In [6]:
connection = connect_H2("jo"+f'{port}',"ba","ba")

Connection to H2 DB successful


## Créer les tables et charger les données bd_jo_v2_H2.sql

In [7]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

NameError: name 'execute' is not defined

## Requêtes

### **Q1**. Les athlètes français (nom pays = 'France') (104 lignes).

In [41]:
query="""


"""
cursor=execute(connection,query,show=True)


104 rows
nomath          |prenomath       |
----------------------------------
FOURCADE        |Martin          |
LAMY CHAPPUIS   |Jason           |
VAULTIER        |Pierre          |
CHAPUIS         |Jean Frederic   |
BRUNET          |Marie Laure     |
DORIN HABERT    |Marie           |
ANTHONIOZ       |Deborah         |
MISSILLIER      |Steve           |
BOVOLENTA       |Arnaud          |
MARTINOD        |Marie           |
GAILLARD        |Jean Marc       |
DE LE RUE       |Paul-Henri      |
PINTURAULT      |Alexis          |
BEATRIX         |Jean Guillaume  |
DUVILLARD       |Robin           |
MANIFICAT       |Maurice         |
PERRILLAT BOITEU|Ivan            |
JOSSERAND       |Marion          |
RAMOIN          |Tony            |
TRESPEUCH       |Chloe           |
MATTEL          |Coline          |
MIDOL           |Jonathan        |
ROLLAND         |Kevin           |
LACROIX         |Sebastian       |
LIZEROUX        |Julien          |
JOUBERT         |Brian           |
CHATAIGNIER

### **Q2** . Les épreuves (sport, nom d'épreuve, catégorie) triées par nom de sport, puis par nom d'épreuve dans l'ordre inverse du dictionnaire (98 lignes).

In [42]:
query="""


"""

cursor=execute(connection,query,show=True)

98 rows
nomep           |categorie       |nomsp           |
---------------------------------------------------
Snowboard cross |Femmes          |Surf des neiges |
Snowboard cross |Hommes          |Surf des neiges |
Slopestyle      |Femmes          |Surf des neiges |
Slopestyle      |Hommes          |Surf des neiges |
Slalom parallèle|Femmes          |Surf des neiges |
Slalom parallèle|Hommes          |Surf des neiges |
Slalom géant par|Femmes          |Surf des neiges |
Slalom géant par|Hommes          |Surf des neiges |
Halfpipe        |Femmes          |Surf des neiges |
Halfpipe        |Hommes          |Surf des neiges |
Sprint par équip|Femmes          |Ski de fond     |
Sprint par équip|Hommes          |Ski de fond     |
Sprint 1,5km    |Femmes          |Ski de fond     |
Sprint 1,5km    |Hommes          |Ski de fond     |
Skiathlon (7,5km|Femmes          |Ski de fond     |
Skiathlon (15km |Hommes          |Ski de fond     |
Relais 4x5km    |Femmes          |Ski de fond     |
Rela

### **Q3** . Les athlètes ayant participé à une équipe de France (30 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q4**. Les épreuves (sport, épreuve, catégorie) auxquelles participent des équipes (25 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q5**. Le pays qui a gagné, en équipe, la médaille d'or dans l'épreuve de la catégorie 'Femmes' intitulée 'relais 4x6km' du sport 'Biathlon' (Ukraine).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q6** . Les athlètes ayant participé à au moins une épreuve individuelle et au moins une par équipe (372 lignes).

In [ ]:
query="""



"""

cursor=execute(connection,query,show=True)

### **Q7** . Les homonymes (les nom de familles portés par deux athlètes ou plus) (141 lignes)

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q8**. Les athlètes ayant participé à (au moins) deux épreuves individuelles (706 lignes).

In [ ]:
query="""



"""

cursor=execute(connection,query,show=True)

### **Q9**. Les athlètes ayant gagné une médaille dans une épreuve individuelle, mais ayant été disqualifiés
dans une autre (14 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q10**. Les athlètes ayant gagné (au moins) une médaille dans une épreuve individuelle (182 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

**NOT EXISTS / NOT IN**

### **Q11**.  Les noms de pays qui n’ont qu’une seule équipe (9 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q12**. Les athlètes ayant participé à exactement 2 épreuves individuelles (402 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

# TME5: JOINTURES ET IMBRICATION AVEC IN ET EXISTS 

Ce TME utilise les données contenues dans le fichier **bd_jo_v2_H2.sql**


## Requêtes

### **Q1**. Les sports auxquels LESSER Erik a participé en supposant qu'il a participé aux épreuves
individuelles (Ski de fond, Biathlon).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q2**. Les athlètes ayant participé aux épreuves individuelles de (au moins) deux sports différents ( PETROVIC Milanko, TER MORS Jorien, LESSER Erik, PEIFFER Arnd).

In [43]:
query="""


"""

cursor=execute(connection,query,show=True)

4 rows
nomath          |prenomath       |
----------------------------------
LESSER          |Erik            |
PEIFFER         |Arnd            |
PETROVIC        |Milanko         |
TER MORS        |Jorien          |


**NOT EXISTS / NOT IN / IS NULL**

### **Q3**. Les dates de début et de fin des Jeux Olympique 2014 (06/02/2014, 23/02/2014).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q4**. Les pays qui ont des participants de moins de 18 ans ou de plus que 40 au 24-02-2016
(Attention : il faut éliminer les athlètes dont on ne connaît pas la date de naissance) (AUT Autriche, CAN Canada, GER Allemagne, ITA Italie, JPN Japon, NOR Norvège, RUS Russie).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q5**. Les équipes n'ayant pas d'athlète (la base de données ne contient pas l'information sur les participants) (13 lignes)

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q6**. Les équipes qui ont exactement 2 athlètes. Retourner l'équipe en question avec ses deux seuls athlètes (114 lignes).


In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q7**. Les sports qui n'ont pas d'épreuves de catégorie Mixte (12 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### **Q8**. Les athlètes qui ont gagné une médaille d’or (au moins) mais pas de médaille d’argent ni de bronze (53 lignes)

In [51]:
query="""


"""

cursor=execute(connection,query,show=True)

53 rows
aid             |nomath          |prenomath       |
---------------------------------------------------
1               |BJOERNDALEN     |Ole Einar       |
2               |BJOERGEN        |Marit           |
6               |SVENDSEN        |Emil Hegle      |
10              |HAMELIN         |Charles         |
14              |DOMRACHEVA      |Darya           |
15              |COLOGNA         |Dario           |
16              |LOCH            |Felix           |
17              |ZHOU            |Yang            |
23              |MAZE            |Tina            |
28              |KOWALCZYK       |Justyna         |
39              |GEISENBERGER    |Natalie         |
41              |BILODEAU        |Alexandre       |
43              |LIGETY          |Ted             |
44              |SANG-HWA        |Lee             |
45              |STOCH           |Kamil           |
46              |TER MORS        |Jorien          |
47              |GRAABAK         |Joergen         |
48  

### **Q9**. Les athlète(s) qui ont fini dernier d’une épreuve individuelle. Indiquer leur nom, prenom, le nom du sport et de l’épreuve et leur rang. Attention : il faut filtrer les athlètes disqualifiés (73 lignes).

In [52]:
query="""


"""

cursor=execute(connection,query,show=True)

73 rows
aid             |nomath          |prenomath       |rang            |nomep           |nomsp           |
------------------------------------------------------------------------------------------------------
12              |HELLNER         |Marcus          |6               |Sprint 1,5km    |Ski de fond     |
34              |PARK            |Seung-Hi        |3               |500m            |Patinage de vite|
173             |SUMANN          |Christoph       |27              |15km départ grou|Biathlon        |
176             |LI              |Nina            |4               |Saut            |Ski acrobatique |
202             |KOSIR           |Zan             |3               |Slalom géant par|Surf des neiges |
212             |DAJING          |Wu              |4               |1000m           |Patinage de vite|
347             |ZAVARZINA       |Alena           |3               |Slalom géant par|Surf des neiges |
410             |CETINKAYA       |Kelime          |61            

# Fermer la connexion

In [8]:
connection.commit() # implicit avec close
connection.close()

In [ ]:
connection